### 목록
* 준비
    * [사용할 라이브러리 불러오기](#lib)
    * [엘라스틱서치 환경 설정하기](#es_setting)
    * [엘라스틱서치에서 데이터 조회하기](#es_load)
    * [조회한 데이터를 테이블 형태로 만들기](#df)
* Parent Pipeline Aggregation
    * [Derivative Aggregation](#der)
    * [Cumulative Sum Aggregation](#cusum)
    * [Moving Avg Aggregation](#mvavg)
    * [Serial Diff Aggregation](#sd)
---

### 준비

<a name='lib'></a>
#### 사용할 라이브러리 불러오기

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import datetime

<a name='es_setting'></a>
#### 엘라스틱서치 환경 설정하기

In [2]:
# 적당한 값을 넣지 않으면 오류가 납니다
es = Elasticsearch()
es_index = 'test_index'
es_type = 'test_type'

<a name='es_load'></a>
#### 엘라스틱서치에서 데이터 조회하기

In [3]:
x = es.search(
      index='test_index', 
      doc_type='test_type', 
      body={
        '_source' : ['주문시간', '상품분류', '고객나이', '구매사이트'],
        'query' : {
            'match_all' : {}
        },
        'size' : 50
      }
)

<a name='df'></a>
#### 조회한 데이터를 테이블 형태로 만들기

In [4]:
df = pd.DataFrame([x['hits']['hits'][idx]['_source'] for idx, _ in enumerate(x['hits']['hits'])])
df

,고객나이,구매사이트,상품분류,주문시간
0,28,옥션,티셔츠,2017-11-01T05:45:01
1,32,티몬,자켓,2017-11-09T16:58:07
2,35,위메프,자켓,2017-11-06T20:03:26
3,38,GS샵,니트,2017-11-03T18:29:35
4,28,쿠팡,블라우스,2017-11-09T22:37:36
5,30,티몬,티셔츠,2017-11-05T00:41:55
6,41,옥션,원피스,2017-11-06T00:27:44
7,38,쿠팡,가디건,2017-11-09T00:40:16
8,39,11번가,셔츠,2017-11-06T10:28:26
9,39,GS샵,스커트,2017-11-08T02:00:16


### Parent Piepline Aggregation

In [5]:
df.index = pd.to_datetime(df['주문시간'])
df_daily_count = df.resample('D').count().iloc[:, 0]
df_daily_count_ = pd.DataFrame({'주문시간' : df_daily_count.index, '개수' : df_daily_count.values.ravel()})
df_daily_count_[['주문시간', '개수']]

,주문시간,개수
0,2017-11-01,2
1,2017-11-02,4
2,2017-11-03,5
3,2017-11-04,7
4,2017-11-05,4
5,2017-11-06,6
6,2017-11-07,5
7,2017-11-08,4
8,2017-11-09,8
9,2017-11-10,5


<a name="der"></a>
#### Derivative Aggregation 

: (Date) Histogram에서 연속된 두 bucket의 aggregated된 값의 차이를 구한다. (현재 값 - 이전 값)

In [6]:
df_daily_count_diff = df_daily_count.diff()
df_derivative = pd.concat([df_daily_count, df_daily_count_diff], axis=1)
df_derivative.columns.values[0], df_derivative.columns.values[1]  = '일별 판매 개수', '전일 대비 판매 개수 변화'
df_derivative

,일별 판매 개수,전일 대비 판매 개수 변화
주문시간,,
2017-11-01,2,NaN
2017-11-02,4,2.0
2017-11-03,5,1.0
2017-11-04,7,2.0
2017-11-05,4,-3.0
2017-11-06,6,2.0
2017-11-07,5,-1.0
2017-11-08,4,-1.0
2017-11-09,8,4.0


<a name="cusum"></a>

#### Cumulative Sum Aggregation
: (Date) Histogram에서 bucket의 aggregate된 값의 누적합 반환한다

In [7]:
df_daily_count_cumsum = df_daily_count.cumsum()
df_cumsum = pd.concat([df_daily_count, df_daily_count_cumsum], axis=1)
df_cumsum.columns.values[0], df_cumsum.columns.values[1]  = '일별 판매 개수', '누적 판매 개수'
df_cumsum

,일별 판매 개수,누적 판매 개수
주문시간,,
2017-11-01,2,2
2017-11-02,4,6
2017-11-03,5,11
2017-11-04,7,18
2017-11-05,4,22
2017-11-06,6,28
2017-11-07,5,33
2017-11-08,4,37
2017-11-09,8,45


<a name="mvavg"></a>
#### Moving Avg Aggregation

: (Date) Histogram에서 bucket의 aggregate된 값들의 이동평균 반환한다

In [8]:
df_daily_count_rolling = df_daily_count.rolling(window=2).mean()
df_rolling = pd.concat([df_daily_count, df_daily_count_rolling], axis=1)
df_rolling.columns.values[0], df_rolling.columns.values[1]  = '일별 판매 개수', '이틀 간 판매 개수의 이동 평균'
df_rolling

,일별 판매 개수,이틀 간 판매 개수의 이동 평균
주문시간,,
2017-11-01,2,NaN
2017-11-02,4,3.0
2017-11-03,5,4.5
2017-11-04,7,6.0
2017-11-05,4,5.5
2017-11-06,6,5.0
2017-11-07,5,5.5
2017-11-08,4,4.5
2017-11-09,8,6.0


<a name="sd"></a>
#### Serial Diff Aggregation 

: (Date) Histogram에서 현재 bucket의 aggregate된 값과 이전 {n}번째 bucket의 aggregate된 값의 차이를 반환한다 (현재 - 이전)

In [9]:
df_daily_count_diff_ = df_daily_count.diff(periods=2)
df_derivative_ = pd.concat([df_daily_count, df_daily_count_diff_], axis=1)
df_derivative_.columns.values[0], df_derivative_.columns.values[1]  = '일별 판매 개수', '전전일 대비 판매 개수 변화'
df_derivative_

,일별 판매 개수,전전일 대비 판매 개수 변화
주문시간,,
2017-11-01,2,NaN
2017-11-02,4,NaN
2017-11-03,5,3.0
2017-11-04,7,3.0
2017-11-05,4,-1.0
2017-11-06,6,-1.0
2017-11-07,5,1.0
2017-11-08,4,-2.0
2017-11-09,8,3.0
